In [ ]:
!pip install xgboost
!pip install optuna
!pip install --upgrade category_encoders
!pip install pandas
!pip install optuna optuna-dashboard
!pip install hyperopt
import os
import pandas as pd
import numpy as np
import csv
import sklearn
import ast
import xgboost as xgb
import category_encoders as catenc
import optuna
from optuna.pruners import ThresholdPruner
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error, r2_score, classification_report, roc_auc_score, mean_absolute_error
from xgboost import XGBRegressor
from category_encoders.target_encoder import TargetEncoder
from sklearn.model_selection import KFold
from sklearn.preprocessing import MultiLabelBinarizer
from IPython.display import Audio


print("Setup Complete")

In [2]:
X = pd.read_csv(r"C:\Users\sbelu\OneDrive\Desktop\ML\Datasets\Housing Test Competition\X_train.csv")
y = pd.read_csv(r"C:\Users\sbelu\OneDrive\Desktop\ML\Datasets\Housing Test Competition\y_train.csv")
Comp_Dataset = pd.read_csv(r"C:\Users\sbelu\OneDrive\Desktop\ML\Datasets\Housing Test Competition\X_test.csv")
Full_Data = pd.read_csv(r"C:\Users\sbelu\OneDrive\Desktop\ML\Datasets\Housing Test Competition\Full_Data.csv")
#----------------------------------------------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=999)
y_train_log = np.log(y_train)
y_log = np.log(y)
print("Data Read")

Data Read


In [4]:
def objectiveMain(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 800, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.03, 0.1),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 0.1),
        'reg_alpha': trial.suggest_float('reg_alpha', 1, 50),
        'reg_lambda': trial.suggest_float('reg_lambda', 1, 50),
        'min_child_weight': trial.suggest_float('min_child_weight', 1, 10),
        'max_delta_step' : trial.suggest_int('max_delta_step', 1, 20),
        'objective' : 'reg:squarederror',
        'random_state': 999,
        'n_jobs': -1
    }

    model = XGBRegressor(**params)
    error = cross_val_score(model, X, y, cv=4, scoring='neg_root_mean_squared_error')#SQRT OCCURS HERE DO NOT ROOT TWICE!
    return -error.mean()

def objectiveMaindowntop(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 800, 1000),
        #'learning_rate': trial.suggest_float('learning_rate', 0.03, 0.1),
        #'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        #'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        #'gamma': trial.suggest_float('gamma', 0, 0.1),
        #'min_child_weight': trial.suggest_float('min_child_weight', 1, 10),
        'objective' : 'reg:squarederror',
        'random_state': 999,
        'n_jobs': -1
    }

    model = XGBRegressor(**params)
    error = cross_val_score(model, X, y, cv=5, scoring='neg_root_mean_squared_error')#SQRT OCCURS HERE DO NOT ROOT TWICE!
    return -error.mean()

def objectiveLog(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 900, 1200),
        'learning_rate': trial.suggest_float('learning_rate', 0.03, 0.05),
        'subsample': trial.suggest_float('subsample', 0.75, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 0.9),
        'gamma': trial.suggest_float('gamma', 0, 0.1),
        #'reg_lambda': trial.suggest_float('reg_lambda', 0, 10),
        'min_child_weight': trial.suggest_float('min_child_weight', 2.5, 3),
        'objective' : 'reg:squarederror',
        'random_state': 999,
        'n_jobs': -1
    }
    params2 = {
        'n_estimators': trial.suggest_int('n_estimators', 500, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0, 1),
        'subsample': trial.suggest_float('subsample', 0.75, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 1),
        #'reg_lambda': trial.suggest_float('reg_lambda', 0, 10),
        'min_child_weight': trial.suggest_float('min_child_weight', 0, 5),
        'objective' : 'reg:squarederror',
        'random_state': 999,
        'n_jobs': -1
    }

    model = XGBRegressor(**params2)
    model.fit(X_train, y_train_log)
    preds = np.exp(model.predict(X_test))
    error = np.sqrt(mean_squared_error(y_test, preds))
    return error


def objectiveLog2Filtered(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 500, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0, 1),
        'subsample': trial.suggest_float('subsample', 0.75, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 1),
        #'reg_lambda': trial.suggest_float('reg_lambda', 0, 10),
        'min_child_weight': trial.suggest_float('min_child_weight', 0, 5),
        'objective' : 'reg:squarederror',
        'random_state': 999,
        'n_jobs': -1
    }
    sampFrac = trial.suggest_float('sample_frac_500k', 0, 1)
    mask = Full_Data['TARGET'] > 500000
    rows_to_remove = Full_Data[mask].sample(frac=sampFrac, random_state=999)
    Full_Data_Filtered = Full_Data.drop(rows_to_remove.index)
    X_F = Full_Data_Filtered.drop(columns=['TARGET'])
    y_F = Full_Data_Filtered['TARGET']
    X_train_F, X_test_F, y_train_F, y_test_F = train_test_split(X_F, y_F, test_size=0.05, random_state=999)
    y_train_log_F = np.log(y_train_F)
    y_log_F = np.log(y_F)
    model = XGBRegressor(**params)
    model.fit(X_train_F, y_train_log_F)
    preds = np.exp(model.predict(X_test_F))
    error = np.sqrt(mean_squared_error(y_test_F, preds))
    return error


In [ ]:
log_study_V2 = optuna.create_study(study_name="log_study_V2", storage="sqlite:///optuna_study.db", sampler=optuna.samplers.TPESampler(seed=999), pruner=optuna.pruners.SuccessiveHalvingPruner(), direction='minimize', load_if_exists=True)
log_study_V2.optimize(objectiveLog, n_trials=50, show_progress_bar=True)

In [ ]:
log_study_V2_genetic = optuna.create_study(study_name="Log_Study_V2_Genetic", storage="sqlite:///optuna_study.db", sampler=optuna.samplers.NSGAIISampler(population_size=35), pruner=optuna.pruners.SuccessiveHalvingPruner(), direction='minimize', load_if_exists=True)
log_study_V2_genetic.optimize(objectiveLog, n_trials=10, show_progress_bar=True)

In [ ]:
log_study_filtered = optuna.create_study(study_name="log_study_filtered", storage="sqlite:///optuna_study.db", sampler=optuna.samplers.TPESampler(seed=999), pruner=optuna.pruners.SuccessiveHalvingPruner(), direction='minimize', load_if_exists=True)
log_study_filtered.optimize(objectiveLog2Filtered, n_trials=50, show_progress_bar=True)

In [ ]:
log_study_V3 = optuna.create_study(study_name="log_study_V3", storage="sqlite:///optuna_study.db", pruner=ThresholdPruner(upper=29000, interval_steps=5, n_warmup_steps=50), direction='minimize', load_if_exists=True)
log_study_V3.optimize(objectiveLog2, n_trials=50, show_progress_bar=True)

In [ ]:
print(log_study_filtered.best_params)
best_filtered_params = {'n_estimators': 987, 'learning_rate': 0.12636682692699636, 'subsample': 0.8452418849838964, 'colsample_bytree': 0.5962763428259379, 'gamma': 0.0001258880483384411, 'min_child_weight': 3.0381729007049763}

In [ ]:
def makeCompSubmission(BEST_PARAMS, filename):
    model = XGBRegressor(**BEST_PARAMS)
    model.fit(X, y_log)
    preds = np.exp(model.predict(Comp_Dataset))
    eval_preds = np.exp(model.predict(X_test))
    error = np.sqrt(mean_squared_error(y_test, eval_preds))
    submission_df = pd.DataFrame({"ID": range(len(preds)),"TARGET": preds})
    submission_df.to_csv(filename+'.csv', index=False)
    print('Best Params: '+ str(BEST_PARAMS))
    

In [ ]:
makeCompSubmission(best_filtered_params, 'submissionV7')